In [ ]:
import websockets
import pandas as pd 
import numpy as np 
import json 
import ta 
import yfinance as yf 
import matplotlib.pyplot as plt 

In [ ]:
stream = \
websockets.connect('wss://stream.binance.com:9443/stream?streams=adausdt@miniTicker')

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','c']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time,unit='ms')
    return df 

In [ ]:
df = pd.DataFrame()
orderlist = []
open_position = False

async with stream as receiver : 
    while True : 
        data = await receiver.recv()
        data = json.loads(data)['data']
        df = pd.concat([df,createframe(data)]).reset_index(drop=True)
        print(df.tail(1))
        
        if not open_position and ta.momentum.roc(df.Price,30).iloc[-1] >0:
            buyprice = df.iloc[-1].Price
            pos = order(price = buyprice, quantity = 50)
            print(f'BUY ORDER : {pos.getbuy()}')
            open_position = True
            
            
        if open_position : 
            subdf = df[df.Time >= pd.to_datetime(pos.getbuy()[1],unit='ms')]
            # ça s'actualise en base revoir la logique
            if len(subdf)>1:
                subdf['highest'] = subdf.Price.cummax()
                subdf['trailingstop'] = subdf['highest']*0.995
                
                if subdf.iloc[-1].Price < subdf.iloc[-1]['trailingstop'] \
                    or subdf.iloc[-1].Price > buyprice * 1.002:
                    
                    sellprice = subdf.iloc[-1].Price
                    pos.addsell(price = sellprice ,quantity =50)
                    print(f'SELL ORDER : {pos.getsell()}')
                    open_position = False
                    print(f"You made {(sellprice - buyprice)/buyprice} profit")
                    orderlist.append(pos)
                    open_position = False